In [ ]:
import xgboost

In [1]:
# Import necessary packages and data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#pandas-jupyter screen output settings
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



### Data Cleansing for whole data set

In [32]:
#read file
filePath=r"E:/Akxay/GLIM/Kaggle/"
data_train=pd.read_csv(filePath+"train.csv", sep=',')
data_test = pd.read_csv(filePath+"test.csv", sep=',')
macro = pd.read_csv(filePath+"macro.csv", sep=',')
print data_train.shape, data_test.shape, macro.shape

(30471, 292) (7662, 291) (2484, 100)


In [ ]:
############ macro data based on VIF
# From here: https://www.kaggle.com/robertoruiz/sberbank-russian-housing-market/dealing-with-multicollinearity/notebook

feature_macro = ["timestamp","balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

macro = macro[feature_macro]

In [ ]:
##### merge the macro data with train and test dataset

data_train = pd.merge(data_train,macro, on="timestamp", how="left")
data_test = pd.merge(data_test,macro, on="timestamp", how="left")

In [3]:
##### convert timestamp 
data_train.timestamp = data_train.timestamp.astype('datetime64[ns]')
data_test.timestamp = data_test.timestamp.astype('datetime64[ns]')

##### month year
data_train['year'] =data_train.timestamp.dt.year
data_train['month'] =data_train.timestamp.dt.month
data_train['dow'] = data_train.timestamp.dt.dayofweek

data_test['year'] = data_test.timestamp.dt.year
data_test['month'] =data_test.timestamp.dt.month
data_test['dow'] = data_test.timestamp.dt.dayofweek

In [4]:
#count number of NA in each column in training set
NA_count=data_train.isnull().sum()
NA_count=pd.DataFrame(NA_count,columns=['count']).sort_values(by="count",axis=0, 
                                                                  ascending=False, inplace=False, kind='quicksort')
print NA_count

                                       count
hospital_beds_raion                    14441
build_year                             13605
state                                  13559
cafe_avg_price_500                     13281
cafe_sum_500_min_price_avg             13281
cafe_sum_500_max_price_avg             13281
max_floor                               9572
material                                9572
num_room                                9572
kitch_sq                                9572
preschool_quota                         6688
school_quota                            6685
cafe_sum_1000_min_price_avg             6524
cafe_sum_1000_max_price_avg             6524
cafe_avg_price_1000                     6524
life_sq                                 6383
build_count_1946-1970                   4991
build_count_1921-1945                   4991
build_count_before_1920                 4991
build_count_after_1995                  4991
raion_build_count_with_builddate_info   4991
build_coun

In [5]:
#count number of NA in each column in test set
NA_count=data_test.isnull().sum()
NA_count=pd.DataFrame(NA_count,columns=['count']).sort_values(by="count",axis=0, 
                                                                  ascending=False, inplace=False, kind='quicksort')
print NA_count

                                       count
hospital_beds_raion                     3418
cafe_sum_500_max_price_avg              3159
cafe_sum_500_min_price_avg              3159
cafe_avg_price_500                      3159
preschool_quota                         1596
school_quota                            1595
cafe_sum_1000_min_price_avg             1222
cafe_sum_1000_max_price_avg             1222
cafe_avg_price_1000                     1222
build_count_panel                       1218
build_count_block                       1218
build_count_wood                        1218
build_count_frame                       1218
build_count_brick                       1218
build_count_monolith                    1218
build_count_mix                         1218
build_count_foam                        1218
build_count_slag                        1218
raion_build_count_with_builddate_info   1218
build_count_before_1920                 1218
build_count_1921-1945                   1218
build_coun

In [6]:
### ### cleaning full_sq, life_sq, kitch_sq- filling discrepencies with NA
data_train.full_sq.loc[data_train.full_sq <= (data_train.life_sq+data_train.kitch_sq)] = np.nan
data_train.full_sq.loc[data_train.full_sq < data_train.life_sq]=np.nan
data_train.life_sq.loc[data_train.full_sq.isnull()==True]= np.nan
data_train.kitch_sq.loc[data_train.full_sq.isnull()==True]= np.nan

# test data
data_test.full_sq.loc[data_test.full_sq <= (data_test.life_sq+data_test.kitch_sq)] = np.nan
data_test.full_sq.loc[data_test.full_sq < data_test.life_sq]=np.nan
data_test.life_sq.loc[data_test.full_sq.isnull()==True]= np.nan
data_test.kitch_sq.loc[data_test.full_sq.isnull()==True]= np.nan

C:\Users\akxay\Anaconda\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
#####Area wise life_sq median is divided by median no of room in NUM_room to achive no of room in missing values of num room
avg_life_sq =pd.DataFrame((data_train.groupby(["sub_area"]).median()).life_sq)
avg_life_sq =avg_life_sq.reset_index()

##### divide life_sq by the overall median num_room (2)
avg_life_sq.life_sq = (avg_life_sq.life_sq/2).round(0)
avg_life_sq =avg_life_sq.rename(columns={'life_sq': 'area_wise_room_size'})
data_train =pd.merge(data_train,avg_life_sq, on=["sub_area"],how="inner")

#test data
#####Area wise life_sq median is divided by median no of room in NUM_room to achive no of room in missing values of num room
avg_life_sq =pd.DataFrame((data_test.groupby(["sub_area"]).median()).life_sq)
avg_life_sq =avg_life_sq.reset_index()

##### divide life_sq by the overall median num_room (2)
avg_life_sq.life_sq = (avg_life_sq.life_sq/2).round(0)
avg_life_sq =avg_life_sq.rename(columns={'life_sq': 'area_wise_room_size'})
data_test =pd.merge(data_test,avg_life_sq, on=["sub_area"],how="inner")

In [8]:
#cleaning life_sq
data_train.loc[(data_train.life_sq < (data_train.num_room*data_train.area_wise_room_size)),"life_sq"]=np.nan

#test data
#cleaning life_sq
data_test.loc[(data_test.life_sq < (data_test.num_room*data_test.area_wise_room_size)),"life_sq"]=np.nan

In [9]:
# data_train = pd.read_csv(r"E:\Akxay\GLIM\Kaggle\file_to_impute.csv")

In [10]:
# while imnputing, recheck all the conditions, if not met, fill it with median
# fill num of rooms with life_sq/15 where 15 is median room size of whole data set
#1.
data_train.loc[(data_train["build_year"]>2018) | (data_train["build_year"]<1900),"build_year"] = np.nan
data_train["build_year"] = data_train["build_year"].apply(lambda x : 2017-x) # execute only once

#test data
# while imnputing, recheck all the conditions, if not met, fill it with median
# fill num of rooms with life_sq/15 where 15 is median room size of whole data set
#1.
data_test.loc[(data_test["build_year"]>2018) | (data_test["build_year"]<1900),"build_year"] = np.nan
data_test["build_year"] = data_test["build_year"].apply(lambda x : 2017-x) # execute only once

In [11]:
#2.
data_train.loc[(data_train["max_floor"]<1 ) | (data_train["floor"] > data_train["max_floor"]),"max_floor"] = np.nan
data_train.loc[(data_train["floor"]<1 ) ,"floor"] = np.nan
data_train.loc[data_train["state"] > 4, "state"] = np.nan

#test data
#2.
data_test.loc[(data_test["max_floor"]<1 ) | (data_test["floor"] > data_test["max_floor"]),"max_floor"] = np.nan
data_test.loc[(data_test["floor"]<1 ) ,"floor"] = np.nan
data_test.loc[data_test["state"] > 4, "state"] = np.nan

In [12]:
data_train = data_train[data_train["full_sq"].notnull()]
data_train = data_train[(data_train["life_sq"].notnull()) | (data_train["kitch_sq"].notnull())]

#test data
data_test = data_test[data_test["full_sq"].notnull()]
data_test = data_test[(data_test["life_sq"].notnull()) | (data_test["kitch_sq"].notnull())]

In [13]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data_test.describe().round(2)

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road2_km,ID_big_road2,railroad_km,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_c

In [14]:
# function to impute kitchen and life sq NA
def fillNA(row,tofill, fromfill):
    if pd.isnull(row[tofill]):
        return row["full_sq"] - row[fromfill]
    else:
        return row[tofill]
data_train["kitch_sq"] = data_train.apply(lambda x: fillNA(x,"kitch_sq","life_sq"), axis=1)
#data_train[data_train["kitch_sq"]==0].shape
data_train["life_sq"] = data_train.apply(lambda x: fillNA(x,"life_sq","kitch_sq"), axis=1)

data_test["kitch_sq"] = data_test.apply(lambda x: fillNA(x,"kitch_sq","life_sq"), axis=1)
#data_test[data_test["kitch_sq"]==0].shape
data_test["life_sq"] = data_test.apply(lambda x: fillNA(x,"life_sq","kitch_sq"), axis=1)

In [15]:
NA_fill_df_train = data_train.groupby(["sub_area","ecology"]).median().iloc[:,1:9]
NA_fill_df_train = NA_fill_df_train.reset_index()

#test data
NA_fill_df_test = data_test.groupby(["sub_area","ecology"]).median().iloc[:,1:9]
NA_fill_df_test = NA_fill_df_test.reset_index()

In [16]:
# data_train = data_train.drop(["Unnamed: 0"],1)

In [17]:
data_train = pd.merge(data_train,NA_fill_df_train, on=["sub_area","ecology"],how="inner")

#test data
data_test = pd.merge(data_test,NA_fill_df_test, on=["sub_area","ecology"],how="inner")

In [18]:
#function to impute NA
def fillNA_by_area_median(row,tofill, fromfill):
    if pd.isnull(row[tofill]):
        return row[fromfill]
    else:
        return row[tofill]

#med_data_df["full_sq_x"] = med_data_df.apply(lambda x: fillNA(x,"full_sq_x","full_sq_y"), axis=1)
# med_data_df["life_sq_x"] = med_data_df.apply(lambda x: fillNA(x,"life_sq_x","life_sq_y"), axis=1)
# med_data_df["max_floor_x"] = med_data_df.apply(lambda x: fillNA(x,"max_floor_x","max_floor_y"), axis=1)
# med_data_df["kitch_sq_x"] = med_data_df.apply(lambda x: fillNA(x,"kitch_sq_x","kitch_sq_y"), axis=1)
data_train["build_year_x"] = data_train.apply(lambda x: fillNA_by_area_median(x,"build_year_x","build_year_y"), axis=1)
data_train["floor_x"] = data_train.apply(lambda x: fillNA_by_area_median(x,"floor_x","floor_y"), axis=1)

#test data
data_test["build_year_x"] = data_test.apply(lambda x: fillNA_by_area_median(x,"build_year_x","build_year_y"), axis=1)
data_test["floor_x"] = data_test.apply(lambda x: fillNA_by_area_median(x,"floor_x","floor_y"), axis=1)

In [19]:
import math
#function to impute NA
def fillNA_for_num_rooms(row,tofill, fromfill):
    if pd.isnull(row[tofill]):
        return math.floor(row["life_sq_x"]/row[fromfill])
    else:
        return row[tofill]
data_train["num_room_x"] = data_train.apply(lambda x: fillNA_for_num_rooms(x,"num_room_x","area_wise_room_size"), axis=1)
#test data
data_test["num_room_x"] = data_test.apply(lambda x: fillNA_for_num_rooms(x,"num_room_x","area_wise_room_size"), axis=1)

In [20]:
data_train = data_train.drop(["max_floor_x","state","hospital_beds_raion","material_x"],1)
#test data
data_test = data_test.drop(["max_floor_x","state","hospital_beds_raion","material_x"],1)

In [21]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data_train.describe().round(2)

,id,full_sq_x,life_sq_x,floor_x,build_year_x,num_room_x,kitch_sq_x,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road2_km,ID_big_road2,railroad_km,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market

In [22]:
#count number of NA in each column
NA_count=data_train.isnull().sum()
NA_count=pd.DataFrame(NA_count,columns=['count']).sort_values(by="count",axis=0, 
                                                                  ascending=False, inplace=False, kind='quicksort')
print NA_count

                                       count
cafe_avg_price_500                     10200
cafe_sum_500_max_price_avg             10200
cafe_sum_500_min_price_avg             10200
cafe_avg_price_1000                     4393
cafe_sum_1000_max_price_avg             4393
cafe_sum_1000_min_price_avg             4393
preschool_quota                         4366
school_quota                            4363
build_count_foam                        3169
raion_build_count_with_material_info    3169
build_count_frame                       3169
build_count_brick                       3169
build_count_monolith                    3169
build_count_panel                       3169
build_count_mix                         3169
build_count_slag                        3169
build_count_block                       3169
raion_build_count_with_builddate_info   3169
build_count_before_1920                 3169
build_count_1921-1945                   3169
build_count_1946-1970                   3169
build_coun

In [23]:
data_train = data_train.drop(["full_sq_y","life_sq_y","floor_y","max_floor_y","material_y","build_year_y","num_room_y","kitch_sq_y",
        "cafe_sum_500_min_price_avg","cafe_avg_price_500", "cafe_sum_500_max_price_avg",
                             "office_sqm_500", "id", "ID_metro", "ID_railroad_station_walk", "ID_railroad_station_avto", 
                              "ID_big_road1", "ID_big_road2", "ID_railroad_terminal","ID_bus_terminal","area_wise_room_size"],1)

data_test = data_test.drop(["full_sq_y","life_sq_y","floor_y","max_floor_y","material_y","build_year_y","num_room_y","kitch_sq_y",
        "cafe_sum_500_min_price_avg","cafe_avg_price_500", "cafe_sum_500_max_price_avg",
                             "office_sqm_500", "id", "ID_metro", "ID_railroad_station_walk", "ID_railroad_station_avto", 
                              "ID_big_road1", "ID_big_road2", "ID_railroad_terminal","ID_bus_terminal","area_wise_room_size"],1)

In [24]:
#fill rest of the columns NA with median
def fillNaWithMedian(columnList):
    for i in columnList:
        data_train[i]=data_train[i].fillna(data_train[i].median())
        data_test[i]=data_test[i].fillna(data_test[i].median())
        
fillNA_withMedian_List=["cafe_sum_1000_max_price_avg","cafe_avg_price_1000","cafe_sum_1000_min_price_avg","preschool_quota",
                        "school_quota","build_count_wood","build_count_after_1995","build_count_1971-1995",
           "build_count_1946-1970","build_count_1921-1945","build_count_before_1920","raion_build_count_with_builddate_info",
                        "build_count_frame", "build_count_mix", "build_count_slag","build_count_block","build_count_panel",
                        "build_count_monolith","build_count_brick", "build_count_foam", "raion_build_count_with_material_info",
                       "cafe_sum_1500_min_price_avg","cafe_avg_price_1500","cafe_sum_1500_max_price_avg","cafe_sum_2000_min_price_avg",
                       "cafe_sum_2000_max_price_avg","cafe_avg_price_2000","cafe_sum_3000_min_price_avg","cafe_avg_price_3000",
                       "cafe_sum_3000_max_price_avg","cafe_sum_5000_min_price_avg","cafe_sum_5000_max_price_avg","cafe_avg_price_5000",
                       "prom_part_5000","metro_km_walk","metro_min_walk","railroad_station_walk_km","railroad_station_walk_min"]

fillNaWithMedian(fillNA_withMedian_List)

# #cafe_sum_1000_max_price_avg             4393
# cafe_avg_price_1000                     4393
# cafe_sum_1000_min_price_avg             4393
# preschool_quota                         4366
# school_quota                            4363
# build_count_wood                        3169
# build_count_after_1995                  3169
# build_count_1971.1995                   3169
# build_count_1946.1970                   3169
# build_count_1921.1945                   3169
# build_count_before_1920                 3169
# raion_build_count_with_builddate_info   3169
# build_count_frame                       3169
# build_count_mix                         3169
# build_count_slag                        3169
# build_count_block                       3169
# build_count_panel                       3169
# build_count_monolith                    3169
# build_count_brick                       3169
# build_count_foam                        3169
# raion_build_count_with_material_info    3169
# cafe_sum_1500_min_price_avg             2578
# cafe_avg_price_1500                     2578
# cafe_sum_1500_max_price_avg             2578
# cafe_sum_2000_min_price_avg             1056
# cafe_sum_2000_max_price_avg             1056
# cafe_avg_price_2000                     1056
# cafe_sum_3000_min_price_avg              707
# cafe_avg_price_3000                      707
# cafe_sum_3000_max_price_avg              707
# cafe_sum_5000_min_price_avg              220
# cafe_sum_5000_max_price_avg              220
# cafe_avg_price_5000                      220
# prom_part_5000                           120
# metro_km_walk                             22
# metro_min_walk                            22
# railroad_station_walk_km                  22
# railroad_station_walk_min                 22


In [25]:
data_test.describe()

,full_sq_x,life_sq_x,floor_x,build_year_x,num_room_x,kitch_sq_x,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,railroad_station_avto_km,railroad_station_avto_min,public_transport_station_km,public_transport_station_min_walk,water_km,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road2_km,railroad_km,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,green_part_500,prom_part_500,office_count_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_

In [26]:
data_train["num_room_x"] = data_train["num_room_x"].astype(np.int64())

In [27]:
y = data_train["price_doc"]
cleaned_data = data_train.drop(["price_doc"],1)

In [28]:
#encoding

cat_X=cleaned_data.select_dtypes(include=['object'])
cat_X_temp=cat_X
num_X=cleaned_data.select_dtypes(include=['float64'])
int64_X=cleaned_data.select_dtypes(include=['int64'])

print cat_X.shape, num_X.shape, int64_X.shape

#---------------------------------------------------------#
#label encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
##le.fit(cat_X)
#le.transform(cat_X) # Encode
cat_X=cat_X.apply(le.fit_transform)

X_all=pd.merge(cat_X,num_X,right_index=True, left_index=True)
X_all=pd.merge(X_all,int64_X,right_index=True, left_index=True)

#test data

cat_X_test=data_test.select_dtypes(include=['object'])
cat_X_test_temp=cat_X_test
num_X_test=data_test.select_dtypes(include=['float64'])
int64_X_test=data_test.select_dtypes(include=['int64'])

print cat_X_test.shape, num_X_test.shape, int64_X_test.shape

#---------------------------------------------------------#
#label encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
##le.fit(cat_X_test)
#le.transform(cat_X_test) # Encode
cat_X_test=cat_X_test.apply(le.fit_transform)

X_all_test=pd.merge(cat_X_test,num_X_test,right_index=True, left_index=True)
X_all_test=pd.merge(X_all_test,int64_X_test,right_index=True, left_index=True)

(26001, 15) (26001, 111) (26001, 151)
(6919, 15) (6919, 111) (6919, 151)


C:\Users\akxay\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [ ]:
###### model

from sklearn import cross_validation
from sklearn.ensemble import RandomForestRegressor


alg=RandomForestRegressor(random_state=1, n_estimators=500,min_samples_split=600,min_samples_leaf=200)
kf = cross_validation.KFold(X_all.shape[0], n_folds=3, random_state=1)
scores = cross_validation.cross_val_score(alg, X_all, y, cv=3)
print scores
print (scores.mean())

In [ ]:
# Important features
alg=RandomForestRegressor(random_state=1, n_estimators=500,min_samples_split=600,min_samples_leaf=200)
forest = alg.fit(X_all,y)
# importances = forest.feature_importances_
# importances

In [ ]:
#feature importance
importances = pd.DataFrame({'feature':X_all.columns,'importance':np.round(forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
print importances
importances.plot.bar()

In [29]:
### XGBOOST
import xgboost as xgb
T_train_xgb = xgb.DMatrix(X_all, y)

params = {"objective": "reg:linear", "booster":"gblinear"}
gbm = xgb.train(dtrain=T_train_xgb,params=params)
print gbm
Y_pred = gbm.predict(xgb.DMatrix(X_all_test))
print Y_pred

[ 3733600.75    5945524.      3888915.75   ..., -1854729.5     1441722.75
  -866744.6875]


In [33]:
data_test.columns

Index([u'id', u'timestamp', u'full_sq', u'life_sq', u'floor', u'max_floor', u'material', u'build_year', u'num_room', u'kitch_sq',
       ...
       u'cafe_count_5000_price_1500', u'cafe_count_5000_price_2500', u'cafe_count_5000_price_4000', u'cafe_count_5000_price_high', u'big_church_count_5000', u'church_count_5000', u'mosque_count_5000', u'leisure_count_5000', u'sport_count_5000', u'market_count_5000'], dtype='object', length=291)

In [36]:
submission = pd.concat([data_test["id"], pd.DataFrame(Y_pred)], axis=1)
submission.to_csv(r"E:\Akxay\GLIM\Kaggle\prediction_xgboost_v2.csv")

In [ ]:
# def lower_bound(dist):
#     iqr = np.percentile(dist, 75) - np.percentile(dist, 25)
#     return np.percentile(dist, 25)-(1.5*iqr)
# def upper_bound(dist):
#     iqr = np.percentile(dist, 75) - np.percentile(dist, 25)
#     return np.percentile(dist, 75)+(1.5*iqr)

In [ ]:
data[data["full_sq"]<lower_bound(data["full_sq"])]["full_sq"]

In [ ]:
(data[data["life_sq"].notnull()]["life_sq"]) 